In [1]:
#SIR Model Infection Rate based on certain properties
import pandas as pd
import networkx as nx
import numpy as np
from source.modules.sir_threaded import sir_model, work, get_accessible_sus_nodes
import source.modules.singleton_reduction as singleton
import source.modules.isolate_reduction as isolate
import copy
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import os
from multiprocessing import Pool
from functools import partial


In [2]:
# Load the full graph
edges = pd.read_csv("data/edges_updated_reversed.csv", sep=' ')
edges_array = np.array(edges.loc[:, ['Source', 'Target']])
nodes = pd.read_csv(r"data/nodes.csv", sep=",")
nodes_array = nodes["# index"]

G_full = nx.DiGraph()
G_full.add_edges_from(edges_array)
G_full.add_nodes_from(nodes_array)

In [3]:
G = copy.deepcopy(G_full)
G = singleton.singleton_reduction(G)
G_full_sr = copy.deepcopy(G)
# isolatedNodes = list(nx.isolates(G))
G, isolatedNum = isolate.isolate_reduction(G)

Node count before singleton reduction:  58742
Nodes before:  52124
Nodes after:  19257


In [ ]:
"""import json
data = json.load(open(r"C:\Users\khatt\Desktop\DependencyBuds\source\positions_all_nodes_excluding_isolated.json"))
pos = {d["node"]: (d["x"], d["y"]) for d in data}"""

In [ ]:
init_infected = [9]
max_steps = 30
G_temp = copy.deepcopy(G)

# Uncomment below block if you want to only view nodes that are accessible from init_infected
G_temp = G_temp.subgraph(get_accessible_sus_nodes(G_temp, init_infected)).copy()
print("Nodes to calculate: ", len(G_temp))
pos = nx.spring_layout(G_temp, seed=8020)

nodelist_total, nodecolors_total, edgecolors_total, options, infotext_total, constants_total = sir_model(G_temp, G_full_sr, init_infected=init_infected, infection_rate=[0.8,0.2], recovery_rate=0, max_steps=max_steps, doSingletonReduction=True, noticeability_rates=(0,0.01), network_type='full')

Nodes to calculate:  4041
Running model...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG] SS:  6618
[DEBUG] SSW:  950
[DEBUG] ST:  4990
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG] SS:  6483
[DEBUG] SSW:  815
[DEBUG] ST:  4290
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG] SS:  6343
[DEBUG] SSW:  675
[DEBUG] ST:  3533
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG] SS:  6229
[DEBUG] SSW:  561
[DEBUG] ST:  2913
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG] SS:  6155
[DEBUG] SSW:  487
[DEBUG] ST:  2490
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG] SS:  6083
[DEBUG] SSW:  415
[DEBUG] ST:  2006
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
~~~

In [22]:
# > Clear all pngs in graphs folder
for filename in os.listdir("graphs"):
   file_path = os.path.join("graphs", filename)
   if os.path.isfile(file_path):
      os.remove(file_path)
        

pos_2 = copy.deepcopy(pos)

worker = partial(
    work,
    G=G_temp,
    pos=pos_2,
    nodelist_total=nodelist_total,
    nodecolors_total=nodecolors_total,
    edgecolors_total=edgecolors_total,
    infotext_total=infotext_total,
    constants_total=constants_total,
    options=options,
)

with Pool(16) as p:
    p.map(worker, range(len(nodelist_total)))

In [23]:
# > Export images in graphs folder to a gif
from PIL import Image

# List of image file paths
images = sorted(os.listdir("graphs"), key=lambda x:int(x[x.find('_')+1:x.find('.')]))

# Open images and store them in a list
frames = [Image.open(f"graphs/{image}") for image in images]

# Save frames as an animated GIF
frames[0].save(
    'sir_model_full_adjusted_recovered.gif',
    save_all=True,
    append_images=frames[1:],
    duration=500,
    loop=0
)